In [30]:
import pandas as pd
import requests
import urllib.parse
from io import StringIO

Yield Data

In [31]:
with open('../keys/api_key.txt', 'r') as file:
    api_key = file.read()

In [32]:
states = ['ILLINOIS', 'INDIANA', 'IOWA', 'MINNESOTA', 'MISSOURI', 'NEBRASKA']

parameters = {
    'key': api_key,
    'source_desc': 'SURVEY',
    'sector_desc': 'CROPS',
    'group_desc': 'FIELD CROPS',
    'commodity_desc': 'CORN',
    'statisticcat_desc': 'YIELD',
    'agg_level_desc': 'STATE',
    'format': 'csv'
}

In [33]:
encoded_parameters = urllib.parse.urlencode(parameters)
base_url = 'https://quickstats.nass.usda.gov/api/api_GET/'
url = base_url+'?'+encoded_parameters
response = requests.get(url)

In [34]:
if response.status_code == 200:
    data = StringIO(response.text)
    yield_raw = pd.read_csv(data)
    yield_raw = yield_raw[['year', 'util_practice_desc', 'state_name', 'reference_period_desc', 'Value']]
    yield_raw = yield_raw[yield_raw['reference_period_desc'] == 'YEAR']
    yield_raw.to_csv('../data/raw/yield_raw.csv', index=False)
    print('Data saved as yield_raw.csv')

else:
    print(f'Request failed with status code {response.status_code}')

Data saved as yield_raw.csv


Price Received Data

In [35]:
for i, state in enumerate(states):

    parameters = {
        'key': api_key,
        'source_desc': 'SURVEY',
        'sector_desc': 'CROPS',
        'group_desc': 'FIELD CROPS',
        'commodity_desc': 'CORN',
        'statisticcat_desc': 'PRICE RECEIVED',
        'agg_level_desc': 'STATE',
        'state_name':state,
        'format': 'csv'
    }

    encoded_parameters = urllib.parse.urlencode(parameters)
    base_url = 'https://quickstats.nass.usda.gov/api/api_GET/'
    url = base_url+'?'+encoded_parameters
    response = requests.get(url)

    if response.status_code == 200:

        if i == 0:
            data = StringIO(response.text)
            price_received_raw = pd.read_csv(data)
            price_received_raw = price_received_raw[['year', 'state_name', 'reference_period_desc', 'Value']]

        else:
            data = StringIO(response.text)
            temp_df = pd.read_csv(data)
            temp_df = temp_df[['year', 'state_name', 'reference_period_desc', 'Value']]
            price_received_raw = pd.concat([price_received_raw, temp_df])

    else:
        print(f'Request failed for {state} with status code {response.status_code}')

price_received_raw.to_csv('../data/raw/price_received_raw.csv', index=False)
print('Data saved as price_received_raw.csv')

Data saved as price_received_raw.csv
